<a href="https://colab.research.google.com/github/Abhishek500/ERA/blob/main/S6/Part2/Abhishekkarn_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 3, padding=1),nn.BatchNorm2d(16),nn.ReLU(),nn.Dropout(0.0))
#         self.conv2 = nn.Sequential(nn.Conv2d(16, 16, 3, padding=1),nn.BatchNorm2d(16),nn.ReLU(),nn.Dropout(0.00))
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=1),nn.BatchNorm2d(32),nn.ReLU(),nn.Dropout(0.00))
#         self.conv4 = nn.Sequential(nn.Conv2d(32, 16, 3, padding=1),nn.BatchNorm2d(16),nn.ReLU(),nn.Dropout(0.00))
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=1),nn.BatchNorm2d(32),nn.ReLU(),nn.Dropout(0.00))
#         self.conv6 = nn.Sequential(nn.Conv2d(32, 4, 3, padding=1),nn.BatchNorm2d(4),nn.ReLU(),nn.Dropout(0.00))
#         self.fc1 = nn.Linear(196, 10)


#     def forward(self, x):
#         x = self.pool1(self.conv2(self.conv1(x)))
#         x = self.pool2(self.conv4(self.conv3(x)))
#         x = self.conv6(self.conv5(x))
#         x = x.view(-1, 196)
#         x = self.fc1(x)

#         #x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [3]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=1), nn.BatchNorm2d(8), nn.ReLU(), nn.Dropout(0.0))
        self.conv2 = nn.Sequential(nn.Conv2d(8, 8, 3), nn.BatchNorm2d(8), nn.ReLU(), nn.Dropout(0.00))
        self.conv3 = nn.Sequential(nn.Conv2d(8, 16, 1), nn.BatchNorm2d(16), nn.ReLU(), nn.Dropout(0.00))
        self.conv4 = nn.Sequential(nn.Conv2d(16, 16, 3), nn.BatchNorm2d(16), nn.ReLU(), nn.Dropout(0.00))
        self.conv5 = nn.Sequential(nn.Conv2d(16, 16, 3), nn.BatchNorm2d(16), nn.ReLU(), nn.Dropout(0.01))
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Sequential(nn.Conv2d(16, 32, 3), nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.02))
        self.conv7 = nn.Sequential(nn.Conv2d(32, 32, 3), nn.BatchNorm2d(32), nn.ReLU(), nn.Dropout(0.00))
        self.global_pool = nn.AdaptiveAvgPool2d(1)  # Global average pooling layer

        self.fc1 = nn.Linear(32, 10)  # Update the input size for the FC layer

    def forward(self, x):
        x = self.conv3(self.conv2(self.conv1(x)))
        x= self.pool1(self.conv5(self.conv4(x)))
        # x = self.pool2(self.conv4(self.conv3(x)))
        x = self.conv7(self.conv6(x))
        x = self.global_pool(x)  # Apply global average pooling
        x = x.view(-1, 32)  # Flatten the tensor
        x = self.fc1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5            [-1, 8, 26, 26]             584
       BatchNorm2d-6            [-1, 8, 26, 26]              16
              ReLU-7            [-1, 8, 26, 26]               0
           Dropout-8            [-1, 8, 26, 26]               0
            Conv2d-9           [-1, 16, 26, 26]             144
      BatchNorm2d-10           [-1, 16, 26, 26]              32
             ReLU-11           [-1, 16, 26, 26]               0
          Dropout-12           [-1, 16, 26, 26]               0
    

<ipython-input-3-a59f52434fea>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                     transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.4, verbose=True)

for epoch in range(1, 21):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-a59f52434fea>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.2652016282081604 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.68it/s]



Test set: Average loss: 0.1919, Accuracy: 9379/10000 (93.79%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 2


loss=0.047356341034173965 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.56it/s]



Test set: Average loss: 0.0732, Accuracy: 9790/10000 (97.90%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 3


...


Test set: Average loss: 0.0173, Accuracy: 9948/10000 (99.48%)

Adjusting learning rate of group 0 to 6.4000e-04.
Epoch 19


loss=0.04625461995601654 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.44it/s]



Test set: Average loss: 0.0176, Accuracy: 9948/10000 (99.48%)

Adjusting learning rate of group 0 to 6.4000e-04.
Epoch 20


loss=0.02031160518527031 batch_id=468: 100%|██████████| 469/469 [00:36<00:00, 13.00it/s]



Test set: Average loss: 0.0184, Accuracy: 9947/10000 (99.47%)

Adjusting learning rate of group 0 to 2.5600e-04.
